<a href="https://colab.research.google.com/github/djamoreland/Quantitative-Neuroscience-Moreland/blob/main/Power_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Do a post-hoc power analysis to determine the number of data samples needed to achieve 80% power for a series of possible effect sizes. Notes and tips:

1. Given that the data have already been collected, note that this is technically a "post-hoc" power analysis and is for demonstration purposes only. Normally these analyses must be done before collecting data.

2. Assume that a "data sample" is simply the non-parametric correlation coefficient between pupil diameter and LC activty measured in a given session.

3. Thus, to obtain a null distribution of these data samples, compute the correlation coefficients on simulated data that are independently generated from Poisson (for LC spiking data) and Gaussian (for pupil data)distributions.


4. Then plot n (number of data samples, corresponding to correlation coefficients measured in a single expermental session) needed for 80% power as a function of effect sizes.

5. To compute power, you can use TTestIndPower in Python, or sampsizepwr in Matlab

In [5]:
from scipy.stats import ttest_ind
from scipy.stats import poisson
from scipy.stats import norm
import numpy as np
from statsmodels.stats.power import TTestIndPower

In [25]:
#Made with help from chatgpt

np.random.seed(42)

N_sim = 3000
within_n = 100
LC_mu = 20
Pupil_mu = 40
Pupil_sigma = 1.0
alpha = 0.05
target_power = 0.80

CC_list = []
for i in range(N_sim):
    lc = poisson.rvs(LC_mu, size=within_n)
    pupil = norm.rvs(loc=Pupil_mu, scale=Pupil_sigma, size=within_n)
    CC = np.corrcoef(lc, pupil)[0, 1]
    CC_list.append(CC)

eps = 1e-12
z_arr = np.arctanh(np.clip(CC_list, -1+eps, 1-eps))
sd_z_sim = np.std(z_arr, ddof=1)

effect_sizes = [0.05, 0.1, 0.2, 0.3, 0.4]
power_calc = TTestIndPower()
results = {}

for CC_mean in effect_sizes:
    z_mean = np.arctanh(CC_mean)
    d = z_mean / sd_z_sim
    n_needed = power_calc.solve_power(effect_size=d,
                                      power=target_power,
                                      alpha=alpha,
                                      alternative='two-sided')
    results[CC_mean] = int(np.ceil(n_needed))

print("Sessions needed for 80% power:")
for CC, n in results.items():
    print(f"  If the mean Correlation Coefficient = {CC:.2f}, then {n} sessions are needed to achiece 80% power")



Sessions needed for 80% power:
  If the mean Correlation Coefficient = 0.05, then 69 sessions are needed to achiece 80% power
  If the mean Correlation Coefficient = 0.10, then 18 sessions are needed to achiece 80% power
  If the mean Correlation Coefficient = 0.20, then 6 sessions are needed to achiece 80% power
  If the mean Correlation Coefficient = 0.30, then 4 sessions are needed to achiece 80% power
  If the mean Correlation Coefficient = 0.40, then 3 sessions are needed to achiece 80% power
